<a href="https://colab.research.google.com/github/Victorlouisdg/simulators/blob/main/fluid_2d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
matplotlib.rc('animation', html='jshtml')

In [ ]:
n = amount_of_cells = 32
velocity_grid = np.zeros((n, n, 2))
cell_size = 1.0 / n
dt = 0.01

In [ ]:
velocity_grid[:, :, 0] = 1  # convention, origin top left, x points down, y points right
velocity_grid[0, 0]

In [ ]:
def checkerboard(size, checker_width = 4):
    board = np.zeros((size, size))
    w = checker_width
    for i in range(w):
        for j in range(w):
            board[i::w * 2, j::w * 2] = 1
            board[i + w::w * 2, j + w::w * 2] = 1
    return board

density_grid = checkerboard(n,16)
plt.imshow(density_grid)

In [ ]:
# These functions return the value of the cell itself at the boundaries
def above(i, j):
    return max(i -1, 0), j

def below(i, j):
    return min(i + 1, n - 1), j

def left(i, j):
    return i, max(j - 1, 0)

def right(i, j):
    return i, min(j + 1, n - 1)

def diffuse(grid, viscocity):
    grid_new = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            grid_new[i, j] = grid[i, j] + viscocity * (grid[above(i, j)] + grid[below(i, j)] + grid[left(i, j)] + grid[right(i, j)] - 4 * grid[i, j])
    return grid_new

plt.imshow(diffuse(density_grid, viscocity=0.1))

In [ ]:
diffusion_history = [density_grid]

for i in range(10):
    density_grid_new = diffuse(density_grid, viscocity=0.2)
    diffusion_history.append(density_grid_new)
    density_grid = density_grid_new

In [ ]:
fig = plt.figure(figsize=(5, 5), dpi=100)
ax = fig.add_subplot(111)
plt.close()  # prevents duplicate output 

def animate(i):
    ax.imshow(diffusion_history[i])
    
animation.FuncAnimation(fig, animate, frames=len(diffusion_history))

In [ ]:
# def bilinear_interpolation(x, y, points):
#     '''Interpolate (x,y) from values associated with four points.

#     The four points are a list of four triplets:  (x, y, value).
#     The four points can be in any order.  They should form a rectangle.

#         >>> bilinear_interpolation(12, 5.5,
#         ...                        [(10, 4, 100),
#         ...                         (20, 4, 200),
#         ...                         (10, 6, 150),
#         ...                         (20, 6, 300)])
#         165.0

#     '''
#     # See formula at:  http://en.wikipedia.org/wiki/Bilinear_interpolation

#     points = sorted(points)               # order points by x, then by y
#     (x1, y1, q11), (_x1, y2, q12), (x2, _y1, q21), (_x2, _y2, q22) = points

#     if x1 != _x1 or x2 != _x2 or y1 != _y1 or y2 != _y2:
#         raise ValueError('points do not form a rectangle')
#     # if not x1 <= x <= x2 or not y1 <= y <= y2:
#     #     raise ValueError('(x, y) not within the rectangle')

#     return (q11 * (x2 - x) * (y2 - y) +
#             q21 * (x - x1) * (y2 - y) +
#             q12 * (x2 - x) * (y - y1) +
#             q22 * (x - x1) * (y - y1)
#            ) / ((x2 - x1) * (y2 - y1) + 0.0)

In [ ]:
# class Grid():
#     def __init__(self, cells=32, size=1.0):
#         self.cells = cells
#         self.size = 1.0
#         self.cell_size = size / cells
#         self.values = np.zeros(cells, cells)

#     def index_to_coordinate(i):
#         # This determines the position of the origin and the orientation of the axes
#         # In this case, the origin is in the center of the cell i=0, j=0, and the 
#         # x-axis points down, the y-axis points to the right
#         return i * self.cell_size


# Grid().cell_size

In [ ]:
# def interpolate_bilinear_from_grid(position, grid):
#     x, y = position

#     # Note that i, j are float here!
#     i = x / cell_size
#     j = y / cell_size

#     i0 = int(np.floor(i)) % n
#     i1 = (i0 + 1) % n
#     j0 = int(np.floor(j)) % n
#     j1 = (j0 + 1) % n

#     x0 = (i0 + 1/2) * cell_size
#     x1 = (i1 + 1/2) * cell_size
#     y0 = (j0 + 1/2) * cell_size
#     y1 = (j1 + 1/2) * cell_size

#     points = (
#         (x0, y0, grid[i0, j0]),
#         (x0, y1, grid[i0, j1]),
#         (x1, y0, grid[i1, j0]),
#         (x1, y1, grid[i1, j1])
#     )

#     return bilinear_interpolation(x, y, points)

In [ ]:
plt.imshow(density_grid)

In [ ]:
def position(i ,j):
    return i * cell_size, j * cell_size


def interpolate_bilinear_from_grid(x, y, grid):
    i = x / cell_size
    j = y / cell_size

    # Handling the corners: 
    # Only 1 value: return it
    if i < 0 and j < 0:
        return grid[0, 0]

    if i < 0 and j >= n - 1:
        return grid[0, n - 1]

    if i >= n - 1 and j < 0:
        return grid[n - 1, 0]
    
    if i >= n - 1 and j >= n - 1:
        return grid[n - 1, n - 1]

    i0 = int(np.floor(i))
    i1 = i0 + 1
    i_frac = i - i0 # fractional part of i

    j0 = int(np.floor(j))
    j1 = j0 + 1
    j_frac = j - j0 # fractional part of j

    # Handling the edges: 
    # Only 2 values: linear interpolation
    if i < 0:
        return (1 - j_frac) * grid[0, j0] + j_frac * grid[0, j1]

    if i >= n - 1:
        return (1 - j_frac) * grid[n - 1, j0] + j_frac * grid[n - 1, j1]

    if j < 0:
        return (1 - i_frac) * grid[i0, 0] + i_frac * grid[i1, 0]
    
    if j >= n - 1:
        return (1 - i_frac) * grid[i0, n - 1] + i_frac * grid[i1, n - 1]

    # Bilinear
    # 4 values: bilinear interpolation
    v = grid[i0, j0] * (1 - i_frac) * (1 - j_frac) + \
        grid[i0, j1] * i_frac * (1 - j_frac) + \
        grid[i1, i0] * (1 - i_frac) * j_frac + \
        grid[i1, j1] * i_frac * j_frac

    return v


def advect(quantity, velocity_grid):
    quantity_new = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            p = position(i, j)
            v = velocity_grid[i, j]
            p_back = p - v * dt

            x, y = p_back

            quantity_new[i, j] = interpolate_bilinear_from_grid(x, y, quantity)

            # p = (i + 1/2) * cell_size, (j + 1/2) * cell_size
            # v = velocity_grid[i, j]
            # # print(v)
            # p_backwards = p - v * dt # meters 
            # new_quantity[i, j] = interpolate_bilinear_from_grid(p_backwards, quantity)
    return quantity_new

plt.imshow(advect(density_grid, velocity_grid))

In [ ]:
for i in range(10):
    density_grid_new = advect(density_grid, velocity_grid)

plt.imshow(advect(density_grid, velocity_grid))

In [ ]:
diffusion_history = [density_grid]

for i in range(10):
    density_grid_new = advect(density_grid, velocity_grid)
    diffusion_history.append(density_grid_new)
    density_grid = density_grid_new

In [ ]:
fig = plt.figure(figsize=(5, 5), dpi=100)
ax = fig.add_subplot(111)
plt.close()  # prevents duplicate output 

def animate(i):
    ax.imshow(diffusion_history[i])
    
animation.FuncAnimation(fig, animate, frames=len(diffusion_history))